In [1]:
# -*- coding: utf-8 -*-
"""
Created on Feb 2 00:19:43 2022

@author: PythonCat_879
"""

from aiohttp import ClientSession
from bs4 import BeautifulSoup
import asyncio
import time
import aiohttp
from fake_useragent import UserAgent # 作假標頭 # 不得不說這一行程式碼真的是偉大的發明
import re
import csv



In [2]:
async def main(lv_name,connector,timeout): # main為非同步程式最高級進入點
    # 給定初始資料
    links = list() #  空串列設置
    for original_url in upper_level:
        links.append(original_url)
    async with ClientSession(connector=connector, timeout=timeout, trust_env = True) as session:
        # trust_env = 用於從 HTTP_PROXY、HTTPS_PROXY、WS_PROXY 或 WSS_PROXY 環境變量中提取代理配置的構造函數（均不區分大小寫）：
        
        #為何要用session？
        #要執行大量的 HTTP 要求，建議使用 ClientSession ，該類別不僅會建立 connection pool
        #更實作了 keep-alive 等功能，可以有效提升執行效率
        #利用Python Comprehesion語法，來建立任務清單(Task)，接著
        #一個session內部包含一個連接池。連接重用和保持連接（預設情況下均處於啟用狀態）可能會提高總體性能。
        
        #透過asyncio模組(Module)的gather()方法(Method)進行打包及執行
        #也由於呼叫的gather()為非同步的方法(Method)，所以要加上await關鍵字，來進行非同步等待。
      
        tasks = [asyncio.create_task(fetch(link, session)) for link in links]  #也可以用這個寫法
        return_list = await asyncio.gather(*tasks)  # 打包任務清單及執行 # 回傳一list
                                                                          
        #print(type(return_list))
        # print(return_list)
        # 這邊的名字目前沒辦法透過迴圈設定，因為非同步的關係，可會造成無法執行，目前還在找原因
        path = lv_name # 每一層的名字(Asynchronous_crawler_level_XX.csv) # csv檔案格式。 
        file = open(path, 'w', newline='', encoding='UTF-8')
        writer = csv.writer(file, quoting=csv.QUOTE_ALL,delimiter=';')
        writer.writerows(return_list)
        file.close()
        
        f = open("time_record_" + str(counter+1) + "level.txt", 'w' , encoding='UTF-8')
        print("花費:" + str(float(time.time() - start_time)) + "秒" , file = f)
        f.close()
        print("花費:" + str(float(time.time() - start_time)) + "秒")
        
        # P.S. 參數加上單引號後，資料被打包成元組
        # 加上await進行非同步等待。
        
#定義協程(coroutine)
async def fetch(link, session):
    # 拿取上面定義的session(也就是任務清單)as response
    every_link = []
    try:
        async with session.get(link, headers = headers, ssl=False) as response:  #非同步發送請求 # 加入headers作假標頭
            # 不檢查SSL (不驗證Https，以節省時間)
            try:
                url_list = []
                parent_link = [] # 這邊一定是空的，目前階段只找out_link
                auth = 1.0
                hub = 1.0
                html_body = await asyncio.wait_for(search_link(response),timeout=3) # 1110103 終於想到如何加上timeout...
                # 加上timeout 心得，必須多寫一個協程，讓其可以套用 asyncio.wait_for 方法，如此一來讓原本一單線程中其一物件進入等待。
                # (在這邊就是 html_body)
                # 然後就可以在這邊加上 asyncio.wait_for 使其擁有timeout功能
                await response.text(encoding="utf-8") # 非同步等待

                                                  # 這邊因為要解析文檔，所以使用text方法，如果要讀取影像，可以使用read方法
                                                  # 1110102 加入utf-8編碼
                                                  # 這邊我的理解為，凡是用非同步請求為開頭來接收的函式，其結果都返回後同樣也要非同步接收
                                                  # 也就是 "await" 方法。
                soup = BeautifulSoup(html_body, 'html5lib')  # 解析HTML原始碼  # 指定'html5lib'作為解析器
                # 下面就可以開始解析soup物件。
                #若只使用sp.link 則只會找出第一筆
                #故這邊要使用"find.all"語法
                link_search = soup.find_all("a") #找出所有的 <a> (超連結)
                for i in range(len(link_search)):
                    k = str(link_search[i].get("href"))  #這行一定要記得要轉type！！！轉成str不然有時候會無法切片取值。
                    if k[0:5] == "https": #只留下"https"
                        url_list.append(k)
                #for url in url_list[:1]: # 印出前1 # 測試有沒有get到東西
                    #print(url) # 印出抓取到的網址 # 0205 印出會造成jupyter無法執行，故在程式測試穩定後應關閉
                # 想辦法存成以下格式
                # every_link = ["original_url","parent_links(list)","children_link(list)","auth(int)","hub(int)"]
                every_link.append(link)
                every_link.append(parent_link)
                every_link.append(url_list)
                every_link.append(auth)
                every_link.append(hub)
            except Exception as e:
                print("timeout!",e)
    except Exception as e:
        # print("Exception",e)
        pass
    return every_link
    # 回傳 every_link = ["original_url","parent_links(list)","children_link(list)","auth(int)","hub(int)"]

async def search_link(response):
    html_body =  await response.text(encoding="utf-8") # 非同步等待
    return html_body

def str_to_url(aaa): # aaa 為文字格式。
    bbb = aaa.split(",") # 利用逗號做切分 # bbb為list格式 # 會回傳一個list
    tmp_url_list = [] # 裝整理好的網址
    for index,url in enumerate(bbb):  # 在這bbb串列中做遞迴
        for index2,char in enumerate(url):
            if char == 'h':
                url = url[index2:] 
                break  # 找到 h開頭就跳出
        url = url[:-1] # 去除尾端單引號
        if len(url) != 0: # 不為空才加入
            tmp_url_list.append(url) #有可能加入奇怪網址，但沒關係，下面會刪除
    for index,url in enumerate(tmp_url_list):
        if url[0] != "h": # 刪除開頭不為h的壞網址
            del tmp_url_list[index]
    # 所以我只要再把這些網址NODE化之後再加入CHILDREN串列就好 =)
    return tmp_url_list #回傳整理好的網址



# 手動初始化

In [6]:
# 初始化由此開始

upper_level = []
path = "url_list.txt"
f = open(path, "r",encoding ="utf-8")
level_0 = f.readlines()
for i in level_0:
    upper_level.append(i[:-1])

upper_level = list(set(upper_level))
print(len(upper_level))
counter=0

353


In [4]:
# 已找過第一層之後需要再執行這一格
# 這邊要把前面的所有檔案的網站總數加起來
# 算出網站總數之後，可以再做一個變數，只算他們的outlink總和(因為original_url，已經找過了)，這樣也可以優化演算法的速度。


csv.field_size_limit(500 * 1024 * 1024)
for i in range(8): # 有幾個檔案range裡面就放幾
    this_level_total_url_list = []
    counter = i
    file_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv' 
    with open(file_name, newline='', encoding = "utf-8") as csvfile:
        # 讀取 CSV 檔案內容
        rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
        for row in rows:
            if len(row) == 0: # 空行跳過
                continue
            if row[0][0:4] == "http":
                this_level_total_url_list.append(row[0]) # 原始url也要加進去
            children_list = str_to_url(row[2]) # 每個url的out_link
            for url in children_list:
                if url[0:4] == "http":
                    this_level_total_url_list.append(url)
        upper_level += this_level_total_url_list
        upper_level = list(set(upper_level))
        print(len(upper_level)) # 已經是不重複的了
counter += 1 # 為了初始化 

11788
84517
448551
685202
856207
1121035
1364472
1546464


In [5]:
# 再次確認
print(len(upper_level))
print(counter)

1546464
8


In [6]:
# 先開啟檔案，以便在接下來的非同步協程中寫入
# 檔案寫入格式[["original_url","parent_links","children_link","auth(int)","hub(int)"],[以此類推]]
# Q：為什麼不加id？ A：因為非同步取下來的網址也是無序的。現在排序沒有意義，到時需要進行二分搜尋法之後再將其排序即可。
csv.field_size_limit(500 * 1024 * 1024) # 增加檔案讀取限制。

524288000

In [7]:
# Step1
# 前面這邊每一層要手動修改
tmp_level = this_level_total_url_list  # 1110204 更動 只找那一層的outlink
level_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv' # 這一層要新創的檔案的名字 # csv 檔案
# 作假標頭
ua = UserAgent()
# 設定參數
headers = {'User-Agent': ua.random}
connector = aiohttp.TCPConnector(limit = 600, ssl=False) # 設定最大連線數量 #1110102 跟改限制為0 (也就是不做限制)
timeout =  aiohttp.ClientTimeout(total = 30000*(counter+1), connect = 3600, # total時間要隨網站數量不同而變動
                      sock_connect=3 , sock_read=3) # 1110102發現，原本預設情況下
                                              # aiohttp 使用總共 300 秒（5 分鐘）的超時時間，這意味著整個操作應該在 5 分鐘內完成。
                                              # 與我發現真實情況相符
                                              # 故我猜測此即為有些request無法完成就中斷的原因
                                              # 1110102將timeout 參數設為 32400 秒，9小時，以令程式盡量全部收到回應。
                                              # connect=300，300秒沒過就time out。
# 整理解釋 aiohttp.ClientTimeout 裡面的參數
# 1. total: 總協程執行時間。
# 2. connect ： 如果超過池(pool)連接限制，則連接建立新連接或等待來自池的空閒連接的最大秒數。
#    (問題：如果pool一直都是被占滿，這樣新建連接要刪除pool中的哪個網址？)
#    (目前解決方法：設定sock_connect的時間比connect的時間短，這樣會先結束長時間無回應的網站，pool就不會誤刪。
# 3. sock_connect：The maximal number of seconds for connecting to a peer for a new connection, not given from a pool.
    #(這邊我的理解為給一個新的連接"最多的"時間限制。)
# 4. sock_read： 讀取網頁資料的最大時間限制(我們這邊都只需要讀網址)，故時間讀取時間應該都很小，故保持預設時間即可
start_time = time.time()  #開始執行時間
loop = asyncio.get_event_loop()  #建立事件迴圈(Event Loop)
loop.create_task(main(level_name,connector,timeout))  #執行協程(coroutine)
print("花費:" + str(float(time.time() - start_time)) + "秒")

# Note: 甚麼時候可以結束？
# 當有輸出txt檔案時代表此cell已經跑完了，可以進行下一步驟。

花費:0.0秒
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 
timeout! 
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 'utf-8' codec can't decode byte 0x82 in position 16: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xc4 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xa1 in position 11: invalid start byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation b

timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 'utf-8' codec can't decode byte 0x83 in position 147: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xc4 in position 10: invalid continuation byte
timeout! Timeout on reading data from socket
timeout! 'utf-8' codec can't decode byte 0x80 in position 10: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xc7 in position 88: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't d

Can not load response cookies: Illegal key 'balancer://npop4cluster'


timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec c

Can not load response cookies: Illegal key 'session-/Common/VIP_ePUAP_PRD_WEB-443'


...

timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xb0 in position 1635: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe1 in position 364: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xa7 in position 246: invalid start byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe9 in position 23626: invalid continuation byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xed in position 148: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 'utf-8' codec can't decod

Can not load response cookies: Illegal key 'session-/Common/VIP_ePUAP_PRD_WEB-443'


timeout! 
timeout! 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte
timeout! 
timeout! 
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xb5 in position 11: invalid start byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte


Can not load response cookies: Illegal key 'session-/Common/VIP_ePUAP_PRD_WEB-443'


timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
timeout! 
timeout! 'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte
花費:5219.921558856964秒


In [8]:
len(upper_level) # P.S.有重複網站的長度

1546464

In [9]:
level_name

'Asynchronous_crawler_level_9.csv'

In [8]:
counter = 0
level_name = 'Asynchronous_crawler_level_'+ str((counter+1)) +'.csv' 

In [11]:
# step_2

# 目標：製作記錄檔以及檢查threshold有沒有達成(增加的網站是否少於總可用網站的10%，若是有則停止)
# 讀csv檔案，並以此製作紀錄檔。
start_time = time.time()  #開始執行時間
csv.field_size_limit(500 * 1024 * 1024)
this_level_total_url_list = [] # 重置  this_level_total_url_list

with open(level_name, newline='', encoding = "utf-8") as csvfile:
    # 讀取 CSV 檔案內容
    rows = csv.reader(csvfile, delimiter=';') # 設定分隔號
    # 以迴圈檢查檢查前11筆
    #count = 0
    for row in rows:
        #if count == 11: 
        #    break
        #print(row)
        if len(row) == 0: # 空行跳過
            continue
        #print(type(row)) # <class 'list'>\n",
        #print(row[0]) # <class 'str'> # original url
        if row[0][0:4] == "http":
            this_level_total_url_list.append(row[0]) # 原始url也要加進去
        #print(row[1]) # <class 'str'> # parent list
        #print(row[2]) # <class 'str'> # children list
        # 將所有children_list 整合以計算數量丟出。
        children_list = str_to_url(row[2])
        for url in children_list:
            if url[0:4] == "http":
                this_level_total_url_list.append(url)
        #print(row[3]) # <class 'str'> # auth score
        #print(row[4]) # <class 'str'> # hub score
        #count += 1
note_path = 'level' + str((counter+1)) + 'Note.txt'
fn = open(note_path, 'w' , encoding='UTF-8')


print("此層增加網站重複總數",len(this_level_total_url_list))
print("此層增加網站重複總數",len(this_level_total_url_list), file = fn)
this_level_total_url_set_list = list(set(this_level_total_url_list))
print("此層增加網站'不'重複總數", len(this_level_total_url_set_list))
print("此層增加網站'不'重複總數", len(this_level_total_url_set_list), file = fn)
total_len = this_level_total_url_list + upper_level # 這一層加上一層 # 這邊就是上面的upper_level
total_len = list(set(total_len)) # 去重複
print("總網站不重複個數", len(total_len))
print("總網站不重複個數", len(total_len), file = fn)
# 只計算這一層而已，還需要跟前一層的總數加起來比較才知道有沒有到達threshold
# 再製作不重複網站增加百分比
print("不重複網站增加百分比", (  len(total_len) / len(upper_level)  ) * 100 , "%" )
print("不重複網站增加百分比", (  len(total_len) / len(upper_level)  ) * 100 , "%"  , file = fn) # 這邊要去讀上一個note的總數
print("花費:" + str(float(time.time() - start_time)) + "秒")







# P.S.
# 但是！！！發現一個問題，0203這天發現有記憶體短缺現象，故中途重啟，發現這行程式碼在中斷之後重啟會有問題。
# 因為目前upper_level的總數為"上一層讀取得條列數目"，我並沒有保留"所有的網站記錄"，故前幾層很有可能新的不重複的網站
# 目前在level_note 4之前的檔案接正確，因為在其之前的網站皆是由下列
# upper_level = total_len 繼承其變數，直到level_4 之前，都是連續執行並未中斷。
# 故total_len 唯一個擁有全部不重複全部網站的變數
# 目前解決方法： 需要將所有不重複的變數引用出(也就是total_len，其實已經存在在csv檔中)
# 基於此判斷，我需要將每一層的檔案都讀取進入。
print("花費:" + str(float(time.time() - start_time)) + "秒" , file = fn)

此層增加網站重複總數 17679
此層增加網站'不'重複總數 11719
總網站不重複個數 11788
不重複網站增加百分比 3339.376770538244 %
花費:0.03200793266296387秒


In [13]:
# 檢查前10筆
total_len[:10]

['https://hi.wiktionary.org/w/index.php?title=सहन&oldid=459434',
 'https://www.motorcyclegear.com/street/just_for_women/off_road_gear',
 'https://www.onions-usa.org/onionista/vegetarian-recipes-with-onion/',
 'https://www.the-exponent.com/relief-society-lesson-8-search-me-o-god-and-know-my-heart/',
 'https://www.tresor.economie.gouv.fr/qui-sommes-nous/offre-d-emplois',
 'https://www.hyperlinkinfosystem.com/hire-dedicated-developers-india.htm',
 'https://science.howstuffworks.com/science-vs-myth/strange-creatures/jersey-devil.htm',
 'https://wikimediaukraine.wordpress.com/2016/04/',
 'https://www.techtarget.com/searchapparchitecture/news/252488601/Mendix-looks-to-change-the-low-code-landscape',
 'https://aceproject.org/electoral-advice/']